In [76]:
import ollama
import chromadb
import pandas as pd

In [77]:
with open('data/recycling guide.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
    
data = {'text': lines}
df = pd.DataFrame(data)

df.head()

,text
0,재활용품은 재질별로 분류하여 분리수거함 및 별도의 전용수거함으로 배출한다.
1,대형 폐가전제품은 무상 방문 수거를 신청한다. 소형 폐가전제품도 같이 수거 가능하다.
2,"대형폐기물은 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재..."
3,음식물류폐기물는 RFID 기반 음식물 폐기장을 사용하거나 종량제봉투에 담아 배출한다...
4,"유해폐기물로 구분된 폐전건지, 형광등, 폐의약품은 전용수거함을 이용한다."


In [78]:
df.tail()

,text
243,식품보조제 콜라겐가루가 남았는데 어떻게 버려야 하나요? 일반쓰레기로 버려야 하나요 ...
244,섬유유연제가 걸쭉하니 굳어가네요. 남은 양이 꽤 많은데 하수구에 버리자니 막힐 것 ...
245,다 쓴 프린터잉크 (페카트리지)는 어떻게 배출하나요?? 저희 읍에서는 종량제봉투에 ...
246,최근 아이스팩이 심각한 수질오염을 일으킬 수 있다는 얘기를 접했습니다. 단순히 종량...
247,"택배 올 때 비닐로 된 포장재가 있습니다. 뽁뽁이 말고요~ 연두빛 비닐인데, 산화 ..."


In [79]:
num_rows = df.shape[0]
print("DataFrame의 줄 개수:", num_rows)

DataFrame의 줄 개수: 248


In [80]:
#client = chromadb.Client()
client = chromadb.PersistentClient(path="DB_recyc")
collection = client.get_or_create_collection(name="recycling_ollama")

docs = df["text"].tolist()

for i, d in enumerate(docs):
  response = ollama.embeddings(model="all-minilm", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [81]:
# an example prompt
prompt = "자전거는 어떻게 버려?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="all-minilm"
)

results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=5
)

res = "\n".join(str(item) for item in results['documents'][0])

print(res)

연탄재는 마대에 담아 배출한다.
나무류인 나무행거는 종량제 봉투로 배출한다.
나무류인 주걱은 종량제 봉투로 배출한다.
나무류인 도마는 종량제 봉투로 배출한다.
나무류인 포장재는 종량제 봉투로 배출한다.


In [86]:
sys_prompt = f'''
너는 재활용 전문가로 활동하고 있어.
사람들이 재활용 방법을 물어보면 너는 주어진 Context를 바탕으로 자세하게 답해줘야해.
너는 한글로 대답을 해야돼.

Context: {res}
'''

prompt = '자전거는 어떻게 버려?'

messages = sys_prompt + prompt

output = ollama.generate(
  model="llama3",
  prompt=messages
)

print(output['response'])

🤔

You're asking how to reuse or dispose of various types of wood waste generated from woodworking processes, such as tree trunks (e), wooden planks (e), door frames (), packaging materials (), and other wooden scraps ().

Here are some possible reuses and disposal methods for each:

1. Tree Trunks (): These can be reused as firewood, mulch, or even turned into furniture or decorative items with proper processing.
2. Wooden Planks (): They can be reused as shelving, flooring, or paneling, depending on their condition and type of wood.
3. Door Frames (): These can be disassembled and reused for other purposes, such as building shelves or creating decorative wall art.
4. Door Mats (): While they may not be suitable for reuse, they can still be composted or repurposed into animal bedding.
5. Packaging Materials (): These can usually be recycled with cardboard or other paper products.

As a professional in reuse and recycling, I recommend exploring creative ways to repurpose these wood was